In [1]:
import sys
sys.path.insert(0, "/home/esr/LRMessanlage/pyLRM")
from pathlib import Path 
import parse
import pandas as pd
import datetime
import assign_tools as a
from bokeh.models import ColumnDataSource,Plot, LinearAxis, Grid, DatetimeTickFormatter,Range1d,LabelSet
from bokeh.plotting import figure, show, output_notebook
from bokeh.models.glyphs import Segment
import logging

logger = logging

output_notebook()


Loading BokehJS ...

### Messung Pfad und name  

In [2]:
path= Path("/home/esr/Documents/Luzern/")
mess_p=Path("/home/esr/Documents/Luzern/Nachmessung2/")
rec_p = path.joinpath("KG")

In [3]:
zeitintervalle = [
    (pd.Timestamp("2018-11-13 07:28:00"),pd.Timestamp("2018-11-16 22:00:00")),
                 ]

zeitintervalle = [
    (pd.Timestamp("2018-11-2 "),pd.Timestamp("2018-11-11 22:00:00")),
                 ]

zeitintervalle = [
    (pd.Timestamp("2018-11-19 "),pd.Timestamp("2018-11-26 22:00:00")),
                 ]

def filt_rec_passby_list(rp_list,zeitintervalle):
    list_f = []
    for pb in rp_list:
        for (t0,t1) in zeitintervalle:
            if pb['start_rec']>t0 and pb['stop_rec']<t1:
                list_f.append(pb)
                break
    return list_f
#between_times= ['8:00', '12:00']

## Lade passby daten

In [4]:
a.test_passby_duration(mess_p,120)

load_passby_info_from_dir: Load passbys from /home/esr/Documents/Luzern/Nachmessung2.
load_passby_info_from_dir: Number of passby found: 1610.
test_passby_duration: 0 passby with duration > than 120.


In [5]:
time_correction=datetime.timedelta(seconds=a.test_passby_time_corrrection(mess_p,9000,20))

load_passby_info_from_dir: Load passbys from /home/esr/Documents/Luzern/Nachmessung2.
load_passby_info_from_dir: Number of passby found: 1610.
test_passby_time_corrrection: Start max correction test:
test_passby_time_corrrection: Start max deviation test:
test_passby_time_corrrection: Stats: 
count    1610.000000
mean    -7233.365506
std         2.115045
min     -7236.953222
25%     -7235.837340
50%     -7232.857239
75%     -7231.500791
max     -7229.830914
dtype: float64


In [6]:
passby_list=a.load_passby_info_from_dir(mess_p,name_match ="*.json")


load_passby_info_from_dir: Load passbys from /home/esr/Documents/Luzern/Nachmessung2.
load_passby_info_from_dir: Number of passby found: 1610.


## Filtere liste anhand datenintervalle 

In [11]:
passby_list_f = filt_rec_passby_list(passby_list,zeitintervalle)
print("Remaining passby: {}.".format(len(passby_list_f)))

NameError: name 'passby_list' is not defined

## Lade XLR rcords 

In [7]:
a.test_XL2_duration(rec_p,120)

load_xl2logs_from_dir: Load xl2 logs from /home/esr/Documents/Luzern/KG.
load_xl2logs_from_dir: Number of XL2 records found: 1436.
test_XL2_duration: 0 XL2 logs with duration > than 120.


In [8]:
rec_list = a.load_xl2logs_from_dir(rec_p)


load_xl2logs_from_dir: Load xl2 logs from /home/esr/Documents/Luzern/KG.
load_xl2logs_from_dir: Number of XL2 records found: 1436.


In [61]:
rec_list_f = filt_rec_passby_list(rec_list,[(t0-time_correction,t1-time_correction) for (t0,t1) in zeitintervalle])
print("Remaining passby: {}.".format(len(rec_list_f)))

Remaining passby: 1191.


In [9]:
assigned,remaining_passby, remaining_rec = a.assign_func(passby_list,rec_list)

assign_func    : Not assigned: Luzern, 2018_11_26_08h15m32s_passby.json.
assign_func    : Not assigned: Luzern, 2018_11_26_07h33m58s_passby.json.
assign_func    : Not assigned: Luzern, 2018_11_25_23h48m21s_passby.json.
assign_func    : Not assigned: Luzern, 2018_11_22_23h19m18s_passby.json.
assign_func    : Not assigned: Luzern, 2018_11_26_04h49m39s_passby.json.
assign_func    : Not assigned: Luzern, 2018_11_22_23h22m24s_passby.json.
assign_func    : Not assigned: Luzern, 2018_11_26_08h12m44s_passby.json.
assign_func    : Not assigned: Luzern, 2018_11_26_09h05m07s_passby.json.
assign_func    : Not assigned: Luzern, 2018_11_24_08h43m26s_passby.json.
assign_func    : Not assigned: Luzern, 2018_11_26_06h06m43s_passby.json.
assign_func    : Not assigned: Luzern, 2018_11_23_06h12m33s_passby.json.
assign_func    : Not assigned: Luzern, 2018_11_22_23h01m46s_passby.json.
assign_func    : Not assigned: Luzern, 2018_11_23_16h45m54s_passby.json.
assign_func    : Not assigned: Luzern, 2018_11_22_2

In [38]:
def plot_zuordnung(assigned,remaining_passby,remaining_rec,labels=True,**kwargs):
    y_range = ["records","passby"]#,"not assigned passby"]
    plt = figure(
        title = "Zuordnung zwische passby und XL2 records.",
        y_range = y_range,
        x_axis_type="datetime",
                 x_axis_label='XL2 uncorrected time',
                 tools=['xwheel_zoom','xpan'], active_scroll='xwheel_zoom',active_drag='xpan',
                **kwargs)
    plt.xaxis.formatter=DatetimeTickFormatter( days=["%m/%d"],months=["%m/%d"], hours=["%m/%d %H:%M"],
                                              hourmin=["%m/%d %H:%M"], minutes=["%m/%d %H:%M"], 
                                              seconds=["%m/%d %H:%M:%S"],minsec=["%m/%d %H:%M:%S"])
    #data
    f_pb = lambda pb:  {'x0':pb['start_rec']-pb['xl2_time_correction'], 'x1':pb['stop_rec']-pb['xl2_time_correction'],'y0':"passby",'y1':"passby",'name':pb['path'].name.replace('_passby.json','')}
    f_rec = lambda r:  {'x0':r['start_rec'], 'x1':r['stop_rec'],'y0':"records",'y1':"records",'name':r['xl2_filename_root']}

    plt.segment(x0="x0", y0="y0", x1="x1", y1="y1",line_width=20,color = "red",
                source = ColumnDataSource(pd.DataFrame([f_pb(pb) for pb in remaining_passby]))
               )
    plt.segment(x0="x0", y0="y0", x1="x1", y1="y1",line_width=20,color = "blue",
                source = ColumnDataSource(pd.DataFrame([f_pb(pb) for pb,r in assigned]))
               )
    #passby
    plt.segment(x0="x0", y0="y0", x1="x1", y1="y1",line_width=20,color = "red",
                source = ColumnDataSource(pd.DataFrame([f_rec(r) for r in remaining_rec]))
               )
    plt.segment(x0="x0", y0="y0", x1="x1", y1="y1",line_width=20,color = "blue",
                source = ColumnDataSource(pd.DataFrame([f_rec(r) for pb,r in assigned]))
               )
    
    if labels:
        plt.add_layout(LabelSet(x='x0', y='y0', text='name', level='glyph', x_offset=5, y_offset=10,
                  source= ColumnDataSource(pd.DataFrame([f_pb(pb) for pb in remaining_passby]))
                               ))
        plt.add_layout(LabelSet(x='x0', y='y0', text='name', level='glyph', x_offset=5, y_offset=10,
                  source= ColumnDataSource(pd.DataFrame([f_rec(r) for r in remaining_rec]))
                      ))

    plt.legend.orientation = "horizontal"
    plt.legend.location = "top_center"
    return plt



In [44]:
import bokeh 
p = plot_zuordnung(assigned,remaining_passby,remaining_rec,width = 1500,height=200)
bokeh.io.save(p,filename=mess_p.absolute().joinpath('plot.html').as_posix(),title='zuordnung_plot')

/home/esr/anaconda3/lib/python3.7/site-packages/bokeh/io/saving.py:123: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")


'/home/esr/Documents/Luzern/Nachmessung2/plot.html'